# 使用LQR算法控制机器人

| Package   | Version |
|-----------|---------|
| numpy | < 2.0 |
| scipy | |
| matplotlib | |
| grpcio | |
| grpcio-tools | |
| torch | |
| 关卡名 | humanoid_test_03 |

* 安装上述软件包，注意 numpy 和 troch 兼容性问题，需要安装 numpy < 2.0 版本



为了使用LQR（线性二次调节器）算法来控制机器人并保持其稳定站立，你需要以下步骤：

1. 建立机器人模型和LQR控制器：首先，你需要建立机器人的状态空间模型，并根据该模型设计LQR控制器。
2. 采集机器人当前状态：使用gRPC接口获取机器人当前的qpos和qvel。
3. 计算控制输入：使用LQR算法计算出控制输入。
4. 发送控制输入：通过gRPC接口发送计算出的控制输入到机器人。

In [ ]:
# 你需要安装 numpy 和 scipy 库用于LQR计算。
# !pip install numpy scipy grpcio grpcio-tools matplotlib


In [ ]:
import os
import sys

current_file_path = os.path.abspath('')
project_root = os.path.dirname(current_file_path)

# 将项目根目录添加到 PYTHONPATH
if project_root not in sys.path:
    sys.path.append(project_root)

import grpc
import importlib
import orca_gym.orca_gym as orca_gym
from orca_gym.orca_gym import OrcaGym


In [ ]:
import asyncio
import nest_asyncio

import numpy as np
import scipy.linalg
from datetime import datetime


# 允许嵌套事件循环 (Nested Event Loops)
nest_asyncio.apply()

config = {
    "fps-30": {
        "frame_time": 0.03333333333333333
    },
    "fps-60": {
        "frame_time": 0.016666666666666666
    },
    "fps-120": {
        "frame_time": 0.008333333333333333
    },
    "fps-2000": {
        "frame_time": 0.0005
    }
}


ctrl0 = None
qpos0 = None

## 寻找最优控制初始化参数

1. 获取模型信息和重量：

* 通过异步gRPC接口查询模型信息和重力配置，计算机器人重量。

2. 设置高精度仿真步长：

* 将仿真步长设置为高精度值以精确找到平衡时的反作用力。

3. 初始化场景和控制器：

* 初始化场景，将控制器设置为零。

3. 记录数据：

* 循环查询位置、速度、反作用力和执行器矩阵，记录每个时间步的数据。

4. 找到平衡帧：

* 通过比较记录的反作用力值，找到垂直反作用力最接近机器人重量的时间步，确定平衡帧。

5. 计算控制力矩：

* 利用平衡帧的反作用力和执行器矩阵的伪逆计算控制力矩，并确保其为负值以反映反作用力。

6. 返回结果：

* 返回平衡帧的位置信息、速度、反作用力和执行器矩阵。

In [ ]:
import matplotlib.pyplot as plt

async def find_balance_frame(sim):

        # 获取模型信息
        model_info = await sim.query_model_info()
        print("Model info: ", model_info)
        nv = model_info['nv']
        

        # 计算小人的重量，获取最佳受力值
        mass = await sim.query_body_subtreemass_by_name("Robot_01_torso")
        opt_config = await sim.query_opt_config()

        gravite = opt_config["gravity"]
        robot_weight = mass * np.linalg.norm(gravite)
        print("Weight: ", robot_weight)

        # 配置仿真步长为 2000 fps，可以精确找到平衡时的反作用力
        await sim.set_opt_timestep(config["fps-2000"]["frame_time"])

        await sim.pause_simulation()

        # 初始化场景
        await sim.load_initial_frame()

        await sim.load_keyframe("stand_on_left_leg")

        qpos, qvel = await sim.query_all_qpos_and_qvel()
        qpos[2] += 0.01  # Offset the height by 0.01.
        await sim.set_qpos(qpos)
    
        # 设置控制器为 0
        await sim.set_ctrl(np.zeros(model_info['nu']))

        await sim.resume_simulation()

        qfrc_inverse_list = []
        qpos_list = []
        qvel_list = []
        actuator_moment_list = []
        frame_count = 7000
        for frame in range(frame_count):            
            # Offset the height by `offset`.
            qpos, qvel = await sim.query_all_qpos_and_qvel()
            # print("Qpos: " , qpos)
            # print("Qpos[2]: " , qpos[2])

            actuator_moment = await sim.query_actuator_moment()

            # print("nv = ", nv, " nu = ", nu)

            await sim.mj_inverse()
            qfrc_inverse = await sim.query_qfrc_inverse()
            # print("Qfrc_inverse[2]: ", qfrc_inverse[2] , " frame = " , frame)
            # print("Qfrc_inverse: ", qfrc_inverse)
            qfrc_inverse_list.append(qfrc_inverse)
            qpos_list.append(qpos)
            qvel_list.append(qvel)
            actuator_moment_list.append(actuator_moment)

        # Find the height-offset at which the vertical force is smallest.
        # 找到qfrc_inverse_2_list中，绝对值最接近weight的值
        qfrc_inverse_list = np.array(qfrc_inverse_list)
        vertical_forces = np.abs(qfrc_inverse_list[:, 2])
        balance_frame = np.argmin(np.abs(vertical_forces - robot_weight))
        # print("Qfrc_inverse 0: ", qfrc_inverse_list[0])
        # print(f"Qfrc_inverse {balance_frame}: ", qfrc_inverse_list[balance_frame])
        # print("Qfrc_inverse 2999: ", qfrc_inverse_list[2999])
        # best_offset = height_offsets[idx]

        # Plot the relationship.
        plt.figure(figsize=(10, 6))
        plt.plot(np.arange(frame_count), vertical_forces, linewidth=3)
        # Red vertical line at offset corresponding to smallest vertical force.
        plt.axvline(x=balance_frame, color='red', linestyle='--')
        # Green horizontal line at the humanoid's weight.

        
        # weight = model.body_subtreemass[1]*np.linalg.norm(model.opt.gravity)

        plt.axhline(y=robot_weight, color='green', linestyle='--')
        plt.xlabel('frames ')
        plt.ylabel('Vertical force (N)')
        plt.grid(which='major', color='#DDDDDD', linewidth=0.8)
        plt.grid(which='minor', color='#EEEEEE', linestyle=':', linewidth=0.5)
        plt.minorticks_on()
        plt.title(f'Found best vertical force at frame {balance_frame}.')
        plt.show()        

        return qpos_list[balance_frame], qvel_list[balance_frame], qfrc_inverse_list[balance_frame], actuator_moment_list[balance_frame]


async def calc_control_0():
    ROBOT_NAMES = ["Robot_01"]
    
    async with orca_gym.grpc.aio.insecure_channel('localhost:50051') as channel:
        stub = orca_gym.mjc_message_pb2_grpc.GrpcServiceStub(channel)
        sim = OrcaGym(stub, ROBOT_NAMES)
    
        await sim.query_agents()

        global ctrl0, qpos0

        # 获取模型信息
        model_info = await sim.query_model_info()

        # 运行一次模拟，获取平衡时的反作用力数组
        qpos0, qvel0, qfrc0, actuator_moment = await find_balance_frame(sim)
        print(f"Balance Frame: qpos0 = {qpos0}, qvel0 = {qvel0}, qfrc0 = {qfrc0}, actuator_moment = {actuator_moment}")
        print(f"Length of qfrc0: {len(qfrc0)}")


        ctrl0 = np.atleast_2d(qfrc0) @ np.linalg.pinv(actuator_moment)
        ctrl0 = ctrl0.flatten()  # Save the ctrl setpoint.
        
        # 转为负值，因为反作用力是机器人对环境的作用力
        ctrl0 = np.multiply(ctrl0, -1) 


        # # Ensure qfrc0 and actuator_moment are compatible for matrix multiplication
        # nv = model_info['nv']
        # qfrc0 = np.array(qfrc0).reshape(nv, 1)  # Reshape qfrc0 to match (nv, 1)
        # actuator_moment_pinv = np.linalg.pinv(actuator_moment)
        
        # print(f"qfrc0 shape: {qfrc0.shape}")
        # print(f"actuator_moment_pinv shape: {actuator_moment_pinv.shape}")
        
        # Perform matrix multiplication
        # ctrl0 = actuator_moment_pinv @ qfrc0  # Ensure correct multiplication order
        # ctrl0 = ctrl0.flatten()  # Save the ctrl setpoint.
        # print('control setpoint:', ctrl0)

        # Set default qpos
        qpos0 = np.array(qpos0)


# if __name__ == "__main__":
#     asyncio.run(calc_control_0())

### Finding the control setpoint using inverse dynamics

MuJoCo's forward dynamics function `mj_forward`, which we used above in order to propagate derived quantities, computes the acceleration given the state and all the forces in the system, some of which are created by the actuators.

The inverse dynamics function takes the acceleration as *input*, and computes the forces required to create the acceleration. Uniquely, MuJoCo's [fast inverse dynamics](https://doi.org/10.1109/ICRA.2014.6907751) takes into account all constraints, including contacts. Let's see how it works.

We'll call the forward dynamics at our desired position setpoint, set the acceleration in `data.qacc` to 0, and call the inverse dynamics:

In [ ]:
async def calc_control_0_by_inverse(floor_offset = 0):    
    async with orca_gym.grpc.aio.insecure_channel('localhost:50051') as channel:
        stub = orca_gym.mjc_message_pb2_grpc.GrpcServiceStub(channel)
        sim = OrcaGym(stub)
    
        model_info = await sim.query_model_info()
        opt_config = await sim.query_opt_config()

        global ctrl0, qpos0

        await sim.pause_simulation()
        await sim.load_initial_frame()

        # mujoco.mj_resetDataKeyframe(model, data, 1)
        # mujoco.mj_forward(model, data)
        # data.qacc = 0  # Assert that there is no the acceleration.
        # mujoco.mj_inverse(model, data)
        # print(data.qfrc_inverse)

        
        await sim.load_keyframe("stand_on_left_leg")
        await sim.mj_forward()
        qacc = np.zeros(model_info['nv'])
        await sim.set_qacc(qacc)
        await sim.mj_inverse()
        qfrc_inverse = await sim.query_qfrc_inverse()
        print("Qfrc_inverse: ", qfrc_inverse)
        

        # height_offsets = np.linspace(-0.001, 0.001, 2001)
        # vertical_forces = []
        # for offset in height_offsets:
        #     mujoco.mj_resetDataKeyframe(model, data, 1)
        #     mujoco.mj_forward(model, data)
        #     data.qacc = 0
        #     # Offset the height by `offset`.
        #     data.qpos[2] += offset
        #     mujoco.mj_inverse(model, data)
        #     vertical_forces.append(data.qfrc_inverse[2])


        height_offsets = np.linspace(-0.001, 0.001, 2001)
        vertical_forces = []
        for offset in height_offsets:
            await sim.load_keyframe("stand_on_left_leg")
            await sim.mj_forward()
            qacc = np.zeros(model_info['nv'])
            await sim.set_qacc(qacc)

            qpos, qvel = await sim.query_all_qpos_and_qvel()
            qpos[2] += offset + floor_offset  # Offset the height by `offset`.
            await sim.set_qpos(qpos)
            await sim.mj_inverse()
            qfrc_inverse = await sim.query_qfrc_inverse()
            vertical_forces.append(qfrc_inverse[2])

        # Find the height-offset at which the vertical force is smallest.
        idx = np.argmin(np.abs(vertical_forces))
        best_offset = height_offsets[idx] + floor_offset
        print(f'Smallest vertical force found {vertical_forces[idx]:.4f}n.')

        # Plot the relationship.
        plt.figure(figsize=(10, 6))
        plt.plot(height_offsets * 1000, vertical_forces, linewidth=3)
        # Red vertical line at offset corresponding to smallest vertical force.
        plt.axvline(x=best_offset*1000, color='red', linestyle='--')
        # Green horizontal line at the humanoid's weight.
        # weight = model.body_subtreemass[1]*np.linalg.norm(model.opt.gravity)
        weight = await sim.query_body_subtreemass_by_name("Robot_01_torso") * np.linalg.norm(opt_config["gravity"])

        plt.axhline(y=weight, color='green', linestyle='--')
        plt.xlabel('Height offset (mm)')
        plt.ylabel('Vertical force (N)')
        plt.grid(which='major', color='#DDDDDD', linewidth=0.8)
        plt.grid(which='minor', color='#EEEEEE', linestyle=':', linewidth=0.5)
        plt.minorticks_on()
        plt.title(f'Smallest vertical force '
                f'found at offset {best_offset*1000:.4f}mm.')
        plt.show()

        # mujoco.mj_resetDataKeyframe(model, data, 1)
        # mujoco.mj_forward(model, data)
        # data.qacc = 0
        # data.qpos[2] += best_offset
        # qpos0 = data.qpos.copy()  # Save the position setpoint.
        # mujoco.mj_inverse(model, data)
        # qfrc0 = data.qfrc_inverse.copy()
        # print('desired forces:', qfrc0)

        await sim.load_keyframe("stand_on_left_leg")
        await sim.mj_forward()
        qacc = np.zeros(model_info['nv'])
        await sim.set_qacc(qacc)
        qpos, qvel = await sim.query_all_qpos_and_qvel()
        qpos[2] += best_offset
        qpos0 = qpos
        await sim.set_qpos(qpos)
        await sim.mj_inverse()
        qfrc0 = await sim.query_qfrc_inverse()
        print('desired forces:', qfrc0)
    
        # ctrl0 = np.atleast_2d(qfrc0) @ np.linalg.pinv(data.actuator_moment)
        # ctrl0 = ctrl0.flatten()  # Save the ctrl setpoint.
        # print('control setpoint:', ctrl0)

        actuator_moment = await sim.query_actuator_moment()
        ctrl0 = np.atleast_2d(qfrc0) @ np.linalg.pinv(actuator_moment)
        ctrl0 = ctrl0.flatten()
        print('control setpoint:', ctrl0)


        # data.ctrl = ctrl0
        # mujoco.mj_forward(model, data)
        # print('actuator forces:', data.qfrc_actuator)

        await sim.set_ctrl(ctrl0)
        await sim.mj_forward()
        qfrc_actuator = await sim.query_qfrc_actuator()
        print('actuator forces:', qfrc_actuator)

        await sim.resume_simulation()

if __name__ == "__main__":
    asyncio.run(calc_control_0_by_inverse(-0.0130))

## 测试一下，运行基于 ctrl0 的控制程序

* 看看小人是不是已经有力气了

In [ ]:
async def test_ctrl0():
    
    async with orca_gym.grpc.aio.insecure_channel('localhost:50051') as channel:
        stub = orca_gym.mjc_message_pb2_grpc.GrpcServiceStub(channel)
        sim = OrcaGym(stub)

        
        global ctrl0
        global qpos0
        # 上面已经计算出了 ctrl0
        print("Control 0: ", ctrl0)
        print("qpos0: ", qpos0)

        mass = await sim.query_body_subtreemass_by_name("Robot_01_torso")
        opt_config = await sim.query_opt_config()        
        print("opt config : ", opt_config)

        # 配置仿真速率为60fps
        await sim.set_opt_timestep(config["fps-60"]["frame_time"])

        # 获取模型信息
        model_info = await sim.query_model_info()
        print("Model info: ", model_info)

        nu = model_info['nu']  # Alias for the number of actuators.
        nv = model_info['nv'] # Shortcut for the number of DoFs.

        # 初始化场景
        await sim.pause_simulation()

        await sim.load_initial_frame()

        await sim.load_keyframe("stand_on_left_leg")

        # await sim.set_qpos(stub, qpos0)
        await sim.set_ctrl(ctrl0)

        await sim.resume_simulation()


if __name__ == "__main__":
    asyncio.run(test_ctrl0())

## 实现LQR控制算法

1. 根据实际的关节顺序，结合原始xml中定义的关键帧，实现关键帧转换，用适配当前场景的关键帧，设置单腿站立
2. 移植mujoco 工程示例代码，实现LQR控制

In [ ]:
async def main():
    
    async with orca_gym.grpc.aio.insecure_channel('localhost:50051') as channel:
        stub = orca_gym.mjc_message_pb2_grpc.GrpcServiceStub(channel)
        sim = OrcaGym(stub)

        mass = await sim.query_body_subtreemass_by_name("Robot_01_torso")
        opt_config = await sim.query_opt_config()        
        print("opt config : ", opt_config)

        frame_time = config["fps-60"]["frame_time"]

        # 配置仿真速率为60fps
        await sim.set_opt_timestep(frame_time)

        # 获取模型信息
        model_info = await sim.query_model_info()
        print("Model info: ", model_info)

        # 获取所有关节信息
        joint_names = await sim.query_joint_names()
        print("Joint names: ", joint_names)

        # 获取所有关节的自由度起始索引
        joint_dofadrs = await sim.query_joint_dofadr(joint_names)
        print("Joint dofadrs: ", joint_dofadrs)

        # 获取控制对象
        actuator_dict = await sim.query_all_actuators()
        print(f"Actuator_info: {actuator_dict}")

        gear_ratios = {key: value['GearRatio'] for key, value in actuator_dict.items()}
        print(f"Gear ratios: {gear_ratios}")

        nu = model_info['nu']  # Alias for the number of actuators.
        R = np.eye(nu)

        nv = model_info['nv'] # Shortcut for the number of DoFs.

        # 初始化场景
        await sim.pause_simulation()
        global ctrl0
        await sim.set_ctrl(ctrl0)

        # init 时候会执行  mj_forward 
        await sim.load_initial_frame()

        await sim.load_keyframe("stand_on_left_leg")
        
        # # 加载 name 为 stand_on_left_leg 的关键帧
        # result = await load_keyframe_to_server(stub, get_qpos_for_keyframe("squat", joint_names, prefix="Robot_01_"))
        # if not result:
        #     print("Failed to load keyframe stand_on_left_leg")
        #     return

        # 用上面找到的平衡位置初始化机器人
        await sim.set_qpos(qpos0)

        # Get the Jacobian for the root body (torso) CoM.        
        await sim.mj_forward()
        jac_com = await sim.mj_jac_subtree_com("Robot_01_torso")
        print("Jacobian CoM: ", jac_com)

        # Get the Jacobian for the left foot.
        jac_foot, _ = await sim.mj_jac_body_com("Robot_01_foot_left")
        print("Jacobian foot: ", jac_foot)

        jac_diff = jac_com - jac_foot
        Qbalance = jac_diff.T @ jac_diff
        print("Qbalance: ", Qbalance)

        # Get indices into relevant sets of joints.
        # 根据关节名称和自由度起始索引生成字典
        joint_dofadr_map = {name: dofadr for name, dofadr in zip(joint_names, joint_dofadrs)}

        # 获取相关关节的自由度索引
        root_dofs = range(6)
        body_dofs = range(6, nv)
        abdomen_dofs = [
            joint_dofadr_map[name]
            for name in joint_names
            if 'abdomen' in name
            and 'z' not in name
        ]
        left_leg_dofs = [
            joint_dofadr_map[name]
            for name in joint_names
            if 'left' in name
            and any(keyword in name for keyword in ['hip', 'knee', 'ankle'])
            and 'z' not in name
        ]
        balance_dofs = abdomen_dofs + left_leg_dofs
        other_dofs = np.setdiff1d(body_dofs, balance_dofs)

        # Cost coefficients.
        BALANCE_COST        = 1000  # Balancing.
        BALANCE_JOINT_COST  = 3     # Joints required for balancing.
        OTHER_JOINT_COST    = .3    # Other joints.

        # Construct the Qjoint matrix.
        Qjoint = np.eye(nv)
        Qjoint[root_dofs, root_dofs] *= 0  # Don't penalize free joint directly.
        Qjoint[balance_dofs, balance_dofs] *= BALANCE_JOINT_COST
        Qjoint[other_dofs, other_dofs] *= OTHER_JOINT_COST

        # Construct the Q matrix for position DoFs.
        Qpos = BALANCE_COST * Qbalance + Qjoint

        # No explicit penalty for velocities.
        Q = np.block([[Qpos, np.zeros((nv, nv))],
                    [np.zeros((nv, 2*nv))]])


        # Set the initial state and control.
        await sim.load_keyframe("stand_on_left_leg")
        await sim.set_qpos(qpos0)
        await sim.set_ctrl(ctrl0)
        await sim.mj_forward()

        # Allocate the A and B matrices, compute them.
        epsilon = 1e-6
        flg_centered = True
        A, B, _, _ = await sim.mjd_transition_fd(epsilon, flg_centered)

        # Solve discrete Riccati equation.
        P = scipy.linalg.solve_discrete_are(A, B, Q, R)

        # Compute the feedback gain matrix K.
        K = np.linalg.inv(R + B.T @ P @ B) @ B.T @ P @ A

        # Allocate position difference dq.
        dq = np.zeros(nv)

        # 循环控制
        for i in range(300):
            start_time = datetime.now()

            # 获取当前关节位置和速度
            qpos, qvel = await sim.query_all_qpos_and_qvel()

            # Get state difference dx.
            dq = await sim.mj_differentiate_pos(qpos0, qpos)
            dx = np.hstack((dq, qvel)).T

            # LQR control law.
            data_ctrl = ctrl0 - K @ dx

            # Set the control signal.
            await sim.set_ctrl(data_ctrl)

            await sim.mj_step(1)

            # 等待到下一次控制周期
            end_time = datetime.now()
            delta_time = (end_time - start_time).total_seconds()
            if delta_time < frame_time:
                await asyncio.sleep(frame_time - delta_time)
            else:
                print(f"Control loop took too long: {delta_time} seconds")

        # 恢复自由执行
        await sim.resume_simulation()

if __name__ == '__main__':
    asyncio.run(main())